In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel('units.xlsx')


In [5]:
df.loc[df.Day == '-', 'Day'] = 7


df.MonthNumber = np.nan
df.loc[df.Month == 'Jan', 'MonthNumber'] = 1
df.loc[df.Month == 'Feb', 'MonthNumber'] = 2
df.loc[df.Month == 'Mar', 'MonthNumber'] = 3
df.loc[df.Month == 'Apr', 'MonthNumber'] = 4
df.loc[df.Month == 'May', 'MonthNumber'] = 5
df.loc[df.Month == 'Jun', 'MonthNumber'] = 6
df.loc[df.Month == 'Jul', 'MonthNumber'] = 7

df  = df.dropna()
raw_data = df

### Q1 : Number of units lost

In [6]:
df.head(3)

,Factory,Month,Week,Day,UnitImpacted,LossCategory,MainRootCause,DetailsOfTheRootCause,Units,SupplierName,ParmaCode,PartReference,Total Loss Cost,MonthNumber
2,VNX,Jan,2.0,1,HDE 11,Blocked_units,Demand change,customer fluctuation,7.0,Cummins Turbo Technologies ( Global ),5385,17467114,7000.0,1.0
3,VNX,Jan,2.0,1,HDE 11,Blocked_units,Demand change,customer fluctuation,13.0,AAM Powertrain Halifax (Global),25333,21722113,13000.0,1.0
5,VNX,Jan,2.0,1,MDE 5,Blocked_units,Production change,production smoothing,1.0,ReflexAllen UK LDT (Global),28803,21769858,1000.0,1.0


In [7]:
df = df.drop(columns=['MainRootCause', 'DetailsOfTheRootCause','SupplierName',
       'ParmaCode', 'PartReference', 'Total Loss Cost'])

In [8]:
target = 'Units'
X = df.drop(columns=[target])
y = df[target]

In [9]:
categorical_features = [column for column in X.columns if X[column].dtype.name == 'object']

# Encode the categorical features using One-hot-encoding
# Hint: you need to specify parameter (of get_dummies) 'columns' using categorical_features
# Implement me
X = pd.get_dummies(data=X, columns=categorical_features)

In [10]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X, y);

In [11]:

predictions_rf = rf.predict(X)

In [12]:

new = pd.DataFrame(np.nan, index=[0],columns=['Week', 'MonthNumber', 'Factory_CUR', 'Factory_HAG', 'Factory_KOP',
       'Factory_SDE', 'Factory_VNX', 'Month_Apr', 'Month_Feb', 'Month_Jan',
       'Month_Jul', 'Month_Jun', 'Month_Mar', 'Month_May', 'Day_1', 'Day_2',
       'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7', 'UnitImpacted_AMT',
       'UnitImpacted_Axles', 'UnitImpacted_DUO', 'UnitImpacted_Duo',
       'UnitImpacted_HDE 11', 'UnitImpacted_HDE 12', 'UnitImpacted_HDE 13',
       'UnitImpacted_HDE 13V', 'UnitImpacted_HDE 16', 'UnitImpacted_HDE 9',
       'UnitImpacted_HDE Penta', 'UnitImpacted_IPS', 'UnitImpacted_MDE 5',
       'UnitImpacted_MDE 8', 'UnitImpacted_PENTA', 'UnitImpacted_Powertronic',
       'UnitImpacted_SMT', 'UnitImpacted_T300', 'LossCategory_Blocked_units',
       'LossCategory_Drumbeats_E', 'LossCategory_Drumbeats_T',
       'LossCategory_Incomplete_units', 'LossCategory_Missing_at_customer',
       'LossCategory_Units_lost'])

In [13]:
CFactory = 'HAG'
CMonth = 'Aug'
CDay = 2
CUnitImpacted = 'HDE 9'
CLossCategory = 'Units_lost'
CMonthNumber = 8
CWeek = 34

In [14]:
Fac_cols = [col for col in new.columns if CFactory in col]
new[Fac_cols] = 1

Mon_cols = [col for col in new.columns if CMonth in col]
new[Mon_cols] = 1

UnitImpacted_cols = [col for col in new.columns if CUnitImpacted in col]
new[UnitImpacted_cols] = 1

LossCategory_cols = [col for col in new.columns if CLossCategory in col]
new[LossCategory_cols] = 1

Day_cols = [col for col in new.columns if str(CDay) in col]
new[Day_cols] = 1

new['MonthNumber'] = CMonthNumber
#new['Day'] = CDay
new['Week'] = CWeek

In [15]:
new  = new.replace(np.nan, 0)

In [16]:
new.head()

,Week,MonthNumber,Factory_CUR,Factory_HAG,Factory_KOP,Factory_SDE,Factory_VNX,Month_Apr,Month_Feb,Month_Jan,...,UnitImpacted_PENTA,UnitImpacted_Powertronic,UnitImpacted_SMT,UnitImpacted_T300,LossCategory_Blocked_units,LossCategory_Drumbeats_E,LossCategory_Drumbeats_T,LossCategory_Incomplete_units,LossCategory_Missing_at_customer,LossCategory_Units_lost
0,34,8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
predictions_rf = rf.predict(new)

In [20]:
round(predictions_rf[0])

5.0

### Q2 : Reasons

In [21]:
q2df = raw_data.drop(columns=['DetailsOfTheRootCause', 'SupplierName',
                              'ParmaCode', 'PartReference', 'Total Loss Cost'])
q2df.loc[q2df.MainRootCause == 'Supplier delay',
         'MainRootCause'] = 'SupplierDelay'
q2df.loc[q2df.MainRootCause == 'Supplier delayed',
         'MainRootCause'] = 'SupplierDelay'

In [22]:

print(q2df.shape)
q2df = q2df.dropna()
print(q2df.shape)

(1202, 9)
(1202, 9)


In [23]:
q2df.head()

,Factory,Month,Week,Day,UnitImpacted,LossCategory,MainRootCause,Units,MonthNumber
2,VNX,Jan,2.0,1,HDE 11,Blocked_units,Demand change,7.0,1.0
3,VNX,Jan,2.0,1,HDE 11,Blocked_units,Demand change,13.0,1.0
5,VNX,Jan,2.0,1,MDE 5,Blocked_units,Production change,1.0,1.0
6,VNX,Jan,2.0,1,MDE 5,Blocked_units,Stock deviation,1.0,1.0
10,VNX,Jan,3.0,1,HDE 11,Blocked_units,Demand change,1.0,1.0


In [24]:
MFactory = 'HAG'
MMonth = 'Aug'
MDay = 2
MUnitImpacted = 'HDE 9'
MLossCategory = 'Units_lost'
MWeek = 35
monthnumber = 8

In [25]:
target_mainrootcause = 'MainRootCause'
X_mainrootcause = q2df.drop(columns=[target_mainrootcause])
y_mainrootcause = q2df[target_mainrootcause]

from sklearn.preprocessing import LabelEncoder
X_mainrootcause = pd.get_dummies(X_mainrootcause, sparse=True)
# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_mainrootcause)
label_encoded_y = label_encoder.transform(y_mainrootcause)

In [26]:
X_mainrootcause.columns

Index(['Week', 'Units', 'MonthNumber', 'Factory_CUR', 'Factory_HAG',
       'Factory_KOP', 'Factory_SDE', 'Factory_VNX', 'Month_Apr', 'Month_Feb',
       'Month_Jan', 'Month_Jul', 'Month_Jun', 'Month_Mar', 'Month_May',
       'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7',
       'UnitImpacted_AMT', 'UnitImpacted_Axles', 'UnitImpacted_DUO',
       'UnitImpacted_Duo', 'UnitImpacted_HDE 11', 'UnitImpacted_HDE 12',
       'UnitImpacted_HDE 13', 'UnitImpacted_HDE 13V', 'UnitImpacted_HDE 16',
       'UnitImpacted_HDE 9', 'UnitImpacted_HDE Penta', 'UnitImpacted_IPS',
       'UnitImpacted_MDE 5', 'UnitImpacted_MDE 8', 'UnitImpacted_PENTA',
       'UnitImpacted_Powertronic', 'UnitImpacted_SMT', 'UnitImpacted_T300',
       'LossCategory_Blocked_units', 'LossCategory_Drumbeats_E',
       'LossCategory_Drumbeats_T', 'LossCategory_Incomplete_units',
       'LossCategory_Missing_at_customer', 'LossCategory_Units_lost'],
      dtype='object')

In [27]:


from sklearn.ensemble import RandomForestClassifier
# Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=100)
# Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_mainrootcause, label_encoded_y)

new = pd.DataFrame(np.nan, index=[0], columns=['Week', 'Units', 'MonthNumber', 'Factory_CUR', 'Factory_HAG',
       'Factory_KOP', 'Factory_SDE', 'Factory_VNX', 'Month_Apr', 'Month_Feb',
       'Month_Jan', 'Month_Jul', 'Month_Jun', 'Month_Mar', 'Month_May',
       'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7',
       'UnitImpacted_AMT', 'UnitImpacted_Axles', 'UnitImpacted_DUO',
       'UnitImpacted_Duo', 'UnitImpacted_HDE 11', 'UnitImpacted_HDE 12',
       'UnitImpacted_HDE 13', 'UnitImpacted_HDE 13V', 'UnitImpacted_HDE 16',
       'UnitImpacted_HDE 9', 'UnitImpacted_HDE Penta', 'UnitImpacted_IPS',
       'UnitImpacted_MDE 5', 'UnitImpacted_MDE 8', 'UnitImpacted_PENTA',
       'UnitImpacted_Powertronic', 'UnitImpacted_SMT', 'UnitImpacted_T300',
       'LossCategory_Blocked_units', 'LossCategory_Drumbeats_E',
       'LossCategory_Drumbeats_T', 'LossCategory_Incomplete_units',
       'LossCategory_Missing_at_customer', 'LossCategory_Units_lost'])

Fac_cols = [col for col in new.columns if MFactory in col]
new[Fac_cols] = 1
Mon_cols = [col for col in new.columns if MMonth in col]
new[Mon_cols] = 1
UnitImpacted_cols = [col for col in new.columns if MUnitImpacted in col]
new[UnitImpacted_cols] = 1
LossCategory_cols = [col for col in new.columns if MLossCategory in col]
new[LossCategory_cols] = 1
Day_cols = [col for col in new.columns if str(MDay) in col]
new[Day_cols] = 1


#new['Day'] = MDay
new['Week'] = MWeek
new['MonthNumber'] = monthnumber
new = new.replace(np.nan, 0)

pred = (clf.predict(new))

In [28]:
label_encoder.inverse_transform(pred)

array(['Internal cause'], dtype=object)

In [29]:
label_encoder.inverse_transform(pred)[0]

'Internal cause'

### Q3 : Lines that are impacted the most

In [211]:
q3df = raw_data.drop(columns=['DetailsOfTheRootCause', 'SupplierName','MainRootCause',
                              'ParmaCode', 'PartReference', 'Total Loss Cost'])
# q3df.loc[q3df.MainRootCause == 'Supplier delay',
#          'MainRootCause'] = 'SupplierDelay'
# q3df.loc[q3df.MainRootCause == 'Supplier delayed',
#          'MainRootCause'] = 'SupplierDelay'

In [212]:
q3df.head()

,Factory,Month,Week,Day,UnitImpacted,LossCategory,Units,MonthNumber
2,VNX,Jan,2.0,1,HDE 11,Blocked_units,7.0,1.0
3,VNX,Jan,2.0,1,HDE 11,Blocked_units,13.0,1.0
5,VNX,Jan,2.0,1,MDE 5,Blocked_units,1.0,1.0
6,VNX,Jan,2.0,1,MDE 5,Blocked_units,1.0,1.0
10,VNX,Jan,3.0,1,HDE 11,Blocked_units,1.0,1.0


In [213]:
LFactory = 'HAG'
LMonth = 'Aug'
LDay = 2
#LUnitImpacted = 'HDE 9'
LLossCategory = 'Units_lost'
LWeek = 35
monthnumber = 8

In [217]:
target_line = 'UnitImpacted'
X_line = q3df.drop(columns=[target_line])
y_line = q3df[target_line]

from sklearn.preprocessing import LabelEncoder
X_line = pd.get_dummies(X_line, sparse=True)
# encode string class values as integers
label_encoder_l = LabelEncoder()
label_encoder_l = label_encoder_l.fit(y_line)
label_encoded_l_y = label_encoder_l.transform(y_line)

In [218]:
X_line.columns

Index(['Week', 'Units', 'MonthNumber', 'Factory_CUR', 'Factory_HAG',
       'Factory_KOP', 'Factory_SDE', 'Factory_VNX', 'Month_Apr', 'Month_Feb',
       'Month_Jan', 'Month_Jul', 'Month_Jun', 'Month_Mar', 'Month_May',
       'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7',
       'LossCategory_Blocked_units', 'LossCategory_Drumbeats_E',
       'LossCategory_Drumbeats_T', 'LossCategory_Incomplete_units',
       'LossCategory_Missing_at_customer', 'LossCategory_Units_lost'],
      dtype='object')

In [220]:


from sklearn.ensemble import RandomForestClassifier
# Create a Gaussian Classifier
clfl = RandomForestClassifier(n_estimators=100)
# Train the model using the training sets y_pred=clf.predict(X_test)
clfl.fit(X_line, label_encoded_l_y)

new = pd.DataFrame(np.nan, index=[0], columns=['Week', 'Units', 'MonthNumber', 'Factory_CUR', 'Factory_HAG',
       'Factory_KOP', 'Factory_SDE', 'Factory_VNX', 'Month_Apr', 'Month_Feb',
       'Month_Jan', 'Month_Jul', 'Month_Jun', 'Month_Mar', 'Month_May',
       'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7',
       'LossCategory_Blocked_units', 'LossCategory_Drumbeats_E',
       'LossCategory_Drumbeats_T', 'LossCategory_Incomplete_units',
       'LossCategory_Missing_at_customer', 'LossCategory_Units_lost'])

Fac_cols = [col for col in new.columns if LFactory in col]
new[Fac_cols] = 1
Mon_cols = [col for col in new.columns if LMonth in col]
new[Mon_cols] = 1
LossCategory_cols = [col for col in new.columns if LLossCategory in col]
new[LossCategory_cols] = 1
Day_cols = [col for col in new.columns if str(MDay) in col]
new[Day_cols] = 1


#new['Day'] = MDay
new['Week'] = MWeek
new['MonthNumber'] = monthnumber
new = new.replace(np.nan, 0)

pred = (clfl.predict(new))

In [221]:
label_encoder_l.inverse_transform(pred)

array(['AMT'], dtype=object)